In [16]:
%display latex

In [17]:
import algorithm_states, common

We start with $ m $ given, and $ a = 1 $.

In [18]:
dim = 6
m0 = random_matrix(ZZ, dim)
m0 = (m0 + m0.transpose()) / 2
(m0, m0.determinant())

(
[    2    -1    -1    -5  -3/2  -3/2]            
[   -1     0     2     0  -3/2    -1]            
[   -1     2     0   3/2     1  -3/2]            
[   -5     0   3/2     7 -11/2     1]            
[ -3/2  -3/2     1 -11/2     1  -3/2]            
[ -3/2    -1  -3/2     1  -3/2   -13], -486513/64
)

Initialization of variables

In [19]:
p = 5
a = 1
state = algorithm_states.ModelState(m0)

If $ \mathop{ord}_p(m_{*1}) > \min_j\{ \mathop{ord}_p(m_{*j}) \} $, there exists $ i $ such that $ \mathop{ord}_p(m_{*i}) = \min_j\{ \mathop{ord}_p(m_{*j}) \} $. We switch the first and $ i $th column (and row), such that the order of the first column is minimal.

In [20]:
(o, i) = min((common.ord_vector(p, state.m, i), i) for i in range(dim))
if o < common.ord_vector(p, state.m, 0):
    state.switch_columns(i, 0)

If $ \mathop{ord}_p(m_{11}) > \mathop{ord}_p(m_{*1}) $ (so $ p \mid p^{-\mathop{ord}_p(m_{*1})} m_{11} $), there exists $ i $ such that $ \mathop{ord}_p(m_{i1}) = \mathop{ord}_p(m_{*1}) $. Then we can add $ 1 $ or $ 2 $ times the $ i $th column and row to the first one, to make sure that $ \mathop{ord}_p(m_{11}) = \mathop{ord}_p(m_{*1}) $.

In [21]:
(o, i) = min((common.ord(p, state.m[i, 0]), i) for i in range(dim))
for j in range(2):
    if o < common.ord(p, state.m[0, 0]):
        state.add_row(i, 0)

If we take $ a $ to be the least common multiple of $ a $ and the numerator of $ m_{11} p^{-\mathop{ord}_p(m_{11})} $, we have for all $ i > 1 $ that $ \frac{m_{1i}}{m_{11}} \in \mathbb Z \left[ \frac 1 a \right] $, so we can add $ -\frac{m_{1i}}{m_{11}} $ the first row and column to the $ i $th one to make sure that in the first row and column, $ m_{11} $ is the only nonzero value.

In [22]:
a = lcm(a, (state.m[0, 0] * p^(-common.ord(p, state.m[0, 0]))).numerator()); a

2

In [23]:
for i in range(1, state.dim):
    state.add_row(0, i, -state.m[0, i] / state.m[0, 0], f"Make entry ({0}, {i}) equal to 0")

Make entry (0, 1) equal to 0


[    2     0    -1    -5  -3/2  -3/2]
[    0  -1/2   3/2  -5/2  -9/4  -7/4]
[   -1   3/2     0   3/2     1  -3/2]
[   -5  -5/2   3/2     7 -11/2     1]
[ -3/2  -9/4     1 -11/2     1  -3/2]
[ -3/2  -7/4  -3/2     1  -3/2   -13]

Make entry (0, 2) equal to 0


[    2     0     0    -5  -3/2  -3/2]
[    0  -1/2   3/2  -5/2  -9/4  -7/4]
[    0   3/2  -1/2    -1   1/4  -9/4]
[   -5  -5/2    -1     7 -11/2     1]
[ -3/2  -9/4   1/4 -11/2     1  -3/2]
[ -3/2  -7/4  -9/4     1  -3/2   -13]

Make entry (0, 3) equal to 0


[    2     0     0     0  -3/2  -3/2]
[    0  -1/2   3/2  -5/2  -9/4  -7/4]
[    0   3/2  -1/2    -1   1/4  -9/4]
[    0  -5/2    -1 -11/2 -37/4 -11/4]
[ -3/2  -9/4   1/4 -37/4     1  -3/2]
[ -3/2  -7/4  -9/4 -11/4  -3/2   -13]

Make entry (0, 4) equal to 0


[    2     0     0     0     0  -3/2]
[    0  -1/2   3/2  -5/2  -9/4  -7/4]
[    0   3/2  -1/2    -1   1/4  -9/4]
[    0  -5/2    -1 -11/2 -37/4 -11/4]
[    0  -9/4   1/4 -37/4  -1/8 -21/8]
[ -3/2  -7/4  -9/4 -11/4 -21/8   -13]

Make entry (0, 5) equal to 0


[     2      0      0      0      0      0]
[     0   -1/2    3/2   -5/2   -9/4   -7/4]
[     0    3/2   -1/2     -1    1/4   -9/4]
[     0   -5/2     -1  -11/2  -37/4  -11/4]
[     0   -9/4    1/4  -37/4   -1/8  -21/8]
[     0   -7/4   -9/4  -11/4  -21/8 -113/8]

If we repeat this for $ m_{22} $ till $ m_{nn} $, by induction, we get the required $ a \in \mathbb Z $ and $ t \in \mathop{SL}_n \left( \mathbb Z \left[ \frac 1 a \right] \right) $ such that $ t^T m t $ is diagonal.

In [24]:
for i in range(1, dim - 1):
    (o, j) = min((common.ord_vector(p, state.m, j), j) for j in range(i, dim))
    if o < common.ord_vector(p, state.m, i):
        state.switch_columns(j, i, description = f"Make sure that the order of column {i} is minimal")
        
    (o, j) = min((common.ord(p, state.m[j, i]), j) for j in range(i, dim))
    for k in range(2):
        if o < common.ord(p, state.m[i, i]):
            state.add_row(j, i, description = f"Make sure that the order of entry ({i}, {i}) is minimal")
            
    a = lcm(a, (state.m[i, i] * p^(-common.ord(p, state.m[i, i]))).numerator())
    
    for j in range(i + 1, state.dim):
        state.add_row(i, j, -state.m[i, j] / state.m[i, i], f"Make entry ({i}, {j}) equal to 0")

Make entry (1, 2) equal to 0


[     2      0      0      0      0      0]
[     0   -1/2      0   -5/2   -9/4   -7/4]
[     0      0      4  -17/2  -13/2  -15/2]
[     0   -5/2  -17/2  -11/2  -37/4  -11/4]
[     0   -9/4  -13/2  -37/4   -1/8  -21/8]
[     0   -7/4  -15/2  -11/4  -21/8 -113/8]

Make entry (1, 3) equal to 0


[     2      0      0      0      0      0]
[     0   -1/2      0      0   -9/4   -7/4]
[     0      0      4  -17/2  -13/2  -15/2]
[     0      0  -17/2      7      2      6]
[     0   -9/4  -13/2      2   -1/8  -21/8]
[     0   -7/4  -15/2      6  -21/8 -113/8]

Make entry (1, 4) equal to 0


...

[      2       0       0       0       0       0]
[      0    -1/2       0       0       0       0]
[      0       0       4       0       0       0]
[      0       0       0 -177/16       0 -159/16]
[      0       0       0       0  711/59 867/236]
[      0       0       0 -159/16 867/236 -353/16]

Make entry (3, 5) equal to 0


[      2       0       0       0       0       0]
[      0    -1/2       0       0       0       0]
[      0       0       4       0       0       0]
[      0       0       0 -177/16       0       0]
[      0       0       0       0  711/59 867/236]
[      0       0       0       0 867/236 -775/59]

Make entry (4, 5) equal to 0


[          2           0           0           0           0           0]
[          0        -1/2           0           0           0           0]
[          0           0           4           0           0           0]
[          0           0           0     -177/16           0           0]
[          0           0           0           0      711/59           0]
[          0           0           0           0           0 -18019/1264]

In [25]:
a.factor()

2^2 * 3^2 * 59 * 79

In [26]:
state.t

[        1       1/2         2      17/4  -329/118 -413/1896]
[        0         1         3      11/8  -129/118   773/632]
[        0         0         1      17/8    -38/59    77/474]
[        0         0         0         1    -63/59  -181/316]
[        0         0         0         0         1  -289/948]
[        0         0         0         0         0         1]

In [27]:
lcm(x.denominator() for x in state.t.coefficients()).factor()

2^3 * 3 * 59 * 79

In [28]:
state.t.transpose() * m0 * state.t

[          2           0           0           0           0           0]
[          0        -1/2           0           0           0           0]
[          0           0           4           0           0           0]
[          0           0           0     -177/16           0           0]
[          0           0           0           0      711/59           0]
[          0           0           0           0           0 -18019/1264]